In [1]:
import Pkg
Pkg.activate(".")

  Activating project at `~/Developer/DistributedStreams`


In [2]:
using Distributed

In [3]:
addprocs(2)

2-element Vector{Int64}:
 2
 3

In [4]:
@everywhere using Base: @kwdef
@everywhere using Distributed, DistributedStreams, DistributedArrays

In [5]:
control_messages, control_responses, distributed_control = launch_sentinel(;
    workers=[2], verbose=true, buffer_size=1024, timeout=1
)

(RemoteChannel{Channel{ControlMessage}}(1, 1, 19), RemoteChannel{Channel{ControlMessage}}(1, 1, 20), @NamedTuple{worker::Int64, flag::Base.RefValue{Bool}}[(worker = 2, flag = Base.RefValue{Bool}(false))])

In [6]:
distributed_control

1-element DArray{@NamedTuple{worker::Int64, flag::Base.RefValue{Bool}}, 1, Vector{@NamedTuple{worker::Int64, flag::Base.RefValue{Bool}}}}:
 (worker = 2, flag = Base.RefValue{Bool}(false))

In [7]:
ch_in = RemoteChannel(()->Channel{Int64}(32), 1)
ch_out = RemoteChannel(()->Channel{Int64}(32), 1)

@everywhere function test_fn(i)
    println("hi there, I'm running on pid=$(myid())")
    sleep(2)
    return i+1
end

In [8]:
m_f = ControlMessage(
    message_type=DistributedStreams.start,
    target=3,
    func_f=test_fn,
    func_in=ch_in,
    func_out=ch_out
)

ControlMessage(DistributedStreams.start, 3, test_fn, RemoteChannel{Channel{Int64}}(1, 1, 27), RemoteChannel{Channel{Int64}}(1, 1, 28))

In [9]:
put!(control_messages, m_f)

RemoteChannel{Channel{ControlMessage}}(1, 1, 19)

      From worker 2:	Start instruction for 3
      From worker 2:	Started


In [10]:
collect!(control_responses)

1-element Vector{ControlMessage}:
 ControlMessage(DistributedStreams.started, 3, nothing, nothing, nothing)

In [21]:
put!(ch_in, 10)

RemoteChannel{Channel{Int64}}(1, 1, 27)

      From worker 3:	hi there, I'm running on pid=3


In [22]:
collect!(ch_out)

1-element Vector{Int64}:
 11

In [17]:
rmprocs(3)

Task (done) @0x00007f3c6ed2ea40

      From worker 2:	Worker 3 died


In [13]:
collect!(control_responses)

ControlMessage[]

In [14]:
addprocs(1)
@everywhere using Distributed, DistributedStreams, DistributedArrays

In [15]:
# ch_in = RemoteChannel(()->Channel{Int64}(32), 1)
# ch_out = RemoteChannel(()->Channel{Int64}(32), 1)

m = ControlMessage(
    message_type=DistributedStreams.start,
    target=4,
    func_f=m_f.func_f,
    func_in=m_f.func_in,
    func_out=m_f.func_out
)

ControlMessage(DistributedStreams.start, 4, test_fn, RemoteChannel{Channel{Int64}}(1, 1, 27), RemoteChannel{Channel{Int64}}(1, 1, 28))

In [16]:
put!(control_messages, m)

RemoteChannel{Channel{ControlMessage}}(1, 1, 19)

      From worker 2:	Start instruction for 4
      From worker 2:	Started


In [17]:
collect!(control_responses)

1-element Vector{ControlMessage}:
 ControlMessage(DistributedStreams.started, 4, nothing, nothing, nothing)

In [23]:
m = ControlMessage(
    message_type=DistributedStreams.stop,
    target=4,
    func_f=nothing,
    func_in=nothing,
    func_out=nothing
)

put!(control_messages, m)

RemoteChannel{Channel{ControlMessage}}(1, 1, 19)

      From worker 2:	Stop instruction for 4
      From worker 2:	Stopped


In [24]:
collect!(control_responses)

1-element Vector{ControlMessage}:
 ControlMessage(DistributedStreams.stopped, 4, nothing, nothing, nothing)

In [25]:
rmprocs(4)

Task (done) @0x00007f3c2ffb5140

In [27]:
collect!(control_responses)

ControlMessage[]

In [28]:
workers()

1-element Vector{Int64}:
 2

In [29]:
stop_workers!(distributed_control; workers=[2])

      From worker 2:	Sentinel on 2 is shutting down


In [30]:
rmprocs()

Task (done) @0x00007f3c2b37a0e0